### Import Dependencies:

conda install pytorch torchvision torchaudio pytorch-cuda=12.1 -c pytorch -c nvidia

conda install -c fastai fastai

In [1]:
from fastai.collab import *
from fastai.tabular.all import *

# Collaborative Filtering Book Recommendation System Using Deep Learning

### Import and Process the Book Data:

### Import Data:

In [2]:
import pandas as pd
ratings = pd.read_csv('../Datasets/kaggle_book_recommendation/Ratings.csv')
books = pd.read_csv('../Datasets/kaggle_book_recommendation/Books.csv', dtype={3: 'str'})

In [11]:
len(ratings)

1149780

In [4]:
ratings['User-ID'].nunique()

105283

In [7]:
ratings['ISBN'].nunique()

340556

In [10]:
books['ISBN'].nunique()

271360

In [14]:
books['Book-Title'].nunique()

242135

### Process the Data:

Merge ratings dataframe with the books dataframe and remove unnecessary columns:

In [20]:
ratings_books_df = ratings.merge(books, on='ISBN')
ratings_books_df.drop(columns=['ISBN', 'Book-Author', 'Year-Of-Publication', 'Publisher', 'Image-URL-S', 'Image-URL-M', 'Image-URL-L'], axis=1, inplace=True)
ratings_books_df.columns = ['user', 'rating', 'title']
ratings_books_df.head()

,user,rating,title
0,276725,0,Flesh Tones: A Novel
1,2313,5,Flesh Tones: A Novel
2,6543,0,Flesh Tones: A Novel
3,8680,5,Flesh Tones: A Novel
4,10314,9,Flesh Tones: A Novel


In [21]:
len(ratings_books_df)

1031136

Filter the Data:

In [43]:
users_with_5_ratings = ratings_books_df.groupby('user').filter(lambda x: len(x) == 5)

In [44]:
df_5 = users_with_5_ratings[users_with_5_ratings['title'].isin(ratings_books_df['title'])]

In [47]:
df_5 = df_5.sort_values(by='user')

In [51]:
df_5

,user,rating,title
188105,183,8,Folio Junior: L'histoire De Monsieur Sommer
188115,183,9,Fahrenheit 451
188121,183,7,Cuentos del Planeta Tierra
188126,183,9,Que Se Mueran Los Feos (Fabula)
188127,183,8,Estudios sobre el amor
...,...,...,...
180703,278668,5,El Alquimista
180709,278668,5,LA Muerte Del Decano
180716,278668,5,Nadie Conoce a Nadie (Tiempos modernos)
180710,278668,5,La Perla del Oriente (ColecciÃ³n Ancora y delfÃ­n)


In [59]:
df_5['comb'] = '"' + df_5['title'] + '" ' + df_5['rating'].astype(str)

In [67]:
df_5

,user,rating,title,comb
188105,183,8,Folio Junior: L'histoire De Monsieur Sommer,"""Folio Junior: L'histoire De Monsieur Sommer"" 8"
188115,183,9,Fahrenheit 451,"""Fahrenheit 451"" 9"
188121,183,7,Cuentos del Planeta Tierra,"""Cuentos del Planeta Tierra"" 7"
188126,183,9,Que Se Mueran Los Feos (Fabula),"""Que Se Mueran Los Feos (Fabula)"" 9"
188127,183,8,Estudios sobre el amor,"""Estudios sobre el amor"" 8"
...,...,...,...,...
180703,278668,5,El Alquimista,"""El Alquimista"" 5"
180709,278668,5,LA Muerte Del Decano,"""LA Muerte Del Decano"" 5"
180716,278668,5,Nadie Conoce a Nadie (Tiempos modernos),"""Nadie Conoce a Nadie (Tiempos modernos)"" 5"
180710,278668,5,La Perla del Oriente (ColecciÃ³n Ancora y delfÃ­n),"""La Perla del Oriente (ColecciÃ³n Ancora y delfÃ­n)"" 5"


In [64]:
# Grouping by 'user' and joining 'comb' values
users_filtered = df_5.groupby('user')['comb'].agg('\n'.join).reset_index()

In [75]:
# Extracting the last sentence from each 'combined' entry
users_filtered['last'] = users_filtered['comb'].apply(lambda x: x.split('\n')[-1])

In [76]:
users_filtered['ans'] = users_filtered['last'].apply(lambda x: x.split(' ')[-1])

In [77]:
users_filtered['query'] = users_filtered['last'].apply(lambda x: ' '.join(x.split()[:-1]))

In [80]:
users_filtered['data'] = users_filtered['comb'].apply(lambda x: '\n'.join(x.split('\n')[:-1]))

In [86]:
users_filtered['prompt'] = users_filtered.apply(lambda row: f"""You are a book recommender and your job is to predict a user's rating (ranging from 1 to 10, with 10 being the highest) on a book, based on that user's previous ratings.

User {row['user']} has rated the following books:

{row['data']}

Predict the user's rating on {row['query']}. Output the rating score only. Do not include other text.""", axis=1)

0                                                                                                                                           You are a book recommender and your job is to predict a user's rating (ranging from 1 to 10, with 10 being the highest) on a book, based on that user's previous ratings.\n\nUser 183 has rated the following books:\n\n"Folio Junior: L'histoire De Monsieur Sommer" 8\n"Fahrenheit 451" 9\n"Cuentos del Planeta Tierra" 7\n"Que Se Mueran Los Feos (Fabula)" 9\n\nPredict the user's rating on "Estudios sobre el amor". Output the rating score only. Do not include other text.
1                                                         You are a book recommender and your job is to predict a user's rating (ranging from 1 to 10, with 10 being the highest) on a book, based on that user's previous ratings.\n\nUser 387 has rated the following books:\n\n"The Adventures of Tom Sawyer" 9\n"Santa Brought A Son : Marrying The Boss's Daughter (Silhouette Romance, 1698)" 6\n"

In [88]:
users_filtered

,user,comb,ans,query,last,data,prompt
0,183,"""Folio Junior: L'histoire De Monsieur Sommer"" 8\n""Fahrenheit 451"" 9\n""Cuentos del Planeta Tierra"" 7\n""Que Se Mueran Los Feos (Fabula)"" 9\n""Estudios sobre el amor"" 8",8,"""Estudios sobre el amor""","""Estudios sobre el amor"" 8","""Folio Junior: L'histoire De Monsieur Sommer"" 8\n""Fahrenheit 451"" 9\n""Cuentos del Planeta Tierra"" 7\n""Que Se Mueran Los Feos (Fabula)"" 9","You are a book recommender and your job is to predict a user's rating (ranging from 1 to 10, with 10 being the highest) on a book, based on that user's previous ratings.\n\nUser 183 has rated the following books:\n\n""Folio Junior: L'histoire De Monsieur Sommer"" 8\n""Fahrenheit 451"" 9\n""Cuentos del Planeta Tierra"" 7\n""Que Se Mueran Los Feos (Fabula)"" 9\n\nPredict the user's rating on ""Estudios sobre el amor"". Output the rating score only. Do not include other text."
1,387,"""The Adventures of Tom Sawyer"" 9\n""Santa Brought A Son : Marrying The Boss's Daughter (Silhouette Romance, 1698)"" 6\n""Alice's Adventures in Wonderland and Through the Looking Glass"" 10\n""Julius Caesar (Oxford School Shakespeare)"" 2\n""Jane Eyre"" 9",9,"""Jane Eyre""","""Jane Eyre"" 9","""The Adventures of Tom Sawyer"" 9\n""Santa Brought A Son : Marrying The Boss's Daughter (Silhouette Romance, 1698)"" 6\n""Alice's Adventures in Wonderland and Through the Looking Glass"" 10\n""Julius Caesar (Oxford School Shakespeare)"" 2","You are a book recommender and your job is to predict a user's rating (ranging from 1 to 10, with 10 being the highest) on a book, based on that user's previous ratings.\n\nUser 387 has rated the following books:\n\n""The Adventures of Tom Sawyer"" 9\n""Santa Brought A Son : Marrying The Boss's Daughter (Silhouette Romance, 1698)"" 6\n""Alice's Adventures in Wonderland and Through the Looking Glass"" 10\n""Julius Caesar (Oxford School Shakespeare)"" 2\n\nPredict the user's rating on ""Jane Eyre"". Output the rating score only. Do not include other text."
2,408,"""Alice's Tulips"" 8\n""Guinness World Records 2000 (Guinness Book of Records, 2000)"" 9\n""Forever Amber Brown (Amber Brown (Paperback))"" 8\n""Feminism and Suffrage: The Emergence of an Independent Women's Movement in America, 1848-1869"" 6\n""Frankenstein (Illustrated Classics Series)"" 7",7,"""Frankenstein (Illustrated Classics Series)""","""Frankenstein (Illustrated Classics Series)"" 7","""Alice's Tulips"" 8\n""Guinness World Records 2000 (Guinness Book of Records, 2000)"" 9\n""Forever Amber Brown (Amber Brown (Paperback))"" 8\n""Feminism and Suffrage: The Emergence of an Independent Women's Movement in America, 1848-1869"" 6","You are a book recommender and your job is to predict a user's rating (ranging from 1 to 10, with 10 being the highest) on a book, based on that user's previous ratings.\n\nUser 408 has rated the following books:\n\n""Alice's Tulips"" 8\n""Guinness World Records 2000 (Guinness Book of Records, 2000)"" 9\n""Forever Amber Brown (Amber Brown (Paperback))"" 8\n""Feminism and Suffrage: The Emergence of an Independent Women's Movement in America, 1848-1869"" 6\n\nPredict the user's rating on ""Frankenstein (Illustrated Classics Series)"". Output the rating score only. Do not include other text."
3,472,"""Organic Chemistry (Saunders Golden Sunburst Series)"" 10\n""Montessori Today: A Comprehensive Approach to Education from Birth to Adulthood"" 9\n""Diet for a Poisoned Planet: How to Choose Safe Foods for You and Your Family"" 9\n""Punished by Rewards: The Trouble With Gold Stars, Incentive Plans, A'S, Praise, and Other Bribes"" 9\n""Secrets of St. John's Wort: A Lynn Sonberg Book"" 5",5,"""Secrets of St. John's Wort: A Lynn Sonberg Book""","""Secrets of St. John's Wort: A Lynn Sonberg Book"" 5","""Organic Chemistry (Saunders Golden Sunburst Series)"" 10\n""Montessori Today: A Comprehensive Approach to Education from Birth to Adulthood"" 9\n""Diet for a Poisoned Planet: How to Choose Safe Foods for You and Your Family"" 9

In [91]:
df_llm_5_items = users_filtered[['prompt', 'query', 'ans']]

In [92]:
df_llm_5_items

,prompt,query,ans
0,"You are a book recommender and your job is to predict a user's rating (ranging from 1 to 10, with 10 being the highest) on a book, based on that user's previous ratings.\n\nUser 183 has rated the following books:\n\n""Folio Junior: L'histoire De Monsieur Sommer"" 8\n""Fahrenheit 451"" 9\n""Cuentos del Planeta Tierra"" 7\n""Que Se Mueran Los Feos (Fabula)"" 9\n\nPredict the user's rating on ""Estudios sobre el amor"". Output the rating score only. Do not include other text.","""Estudios sobre el amor""",8
1,"You are a book recommender and your job is to predict a user's rating (ranging from 1 to 10, with 10 being the highest) on a book, based on that user's previous ratings.\n\nUser 387 has rated the following books:\n\n""The Adventures of Tom Sawyer"" 9\n""Santa Brought A Son : Marrying The Boss's Daughter (Silhouette Romance, 1698)"" 6\n""Alice's Adventures in Wonderland and Through the Looking Glass"" 10\n""Julius Caesar (Oxford School Shakespeare)"" 2\n\nPredict the user's rating on ""Jane Eyre"". Output the rating score only. Do not include other text.","""Jane Eyre""",9
2,"You are a book recommender and your job is to predict a user's rating (ranging from 1 to 10, with 10 being the highest) on a book, based on that user's previous ratings.\n\nUser 408 has rated the following books:\n\n""Alice's Tulips"" 8\n""Guinness World Records 2000 (Guinness Book of Records, 2000)"" 9\n""Forever Amber Brown (Amber Brown (Paperback))"" 8\n""Feminism and Suffrage: The Emergence of an Independent Women's Movement in America, 1848-1869"" 6\n\nPredict the user's rating on ""Frankenstein (Illustrated Classics Series)"". Output the rating score only. Do not include other text.","""Frankenstein (Illustrated Classics Series)""",7
3,"You are a book recommender and your job is to predict a user's rating (ranging from 1 to 10, with 10 being the highest) on a book, based on that user's previous ratings.\n\nUser 472 has rated the following books:\n\n""Organic Chemistry (Saunders Golden Sunburst Series)"" 10\n""Montessori Today: A Comprehensive Approach to Education from Birth to Adulthood"" 9\n""Diet for a Poisoned Planet: How to Choose Safe Foods for You and Your Family"" 9\n""Punished by Rewards: The Trouble With Gold Stars, Incentive Plans, A'S, Praise, and Other Bribes"" 9\n\nPredict the user's rating on ""Secrets of St. John's...","""Secrets of St. John's Wort: A Lynn Sonberg Book""",5
4,"You are a book recommender and your job is to predict a user's rating (ranging from 1 to 10, with 10 being the highest) on a book, based on that user's previous ratings.\n\nUser 566 has rated the following books:\n\n""Who Moved My Cheese? An Amazing Way to Deal with Change in Your Work and in Your Life"" 2\n""Stately Pursuits"" 6\n""Women on the Case"" 6\n""Almost: A Novel"" 8\n\nPredict the user's rating on ""New Cardiff"". Output the rating score only. Do not include other text.","""New Cardiff""",6
...,...,...,...
1998,"You are a book recommender and your job is to predict a user's rating (ranging from 1 to 10, with 10 being the highest) on a book, based on that user's previous ratings.\n\nUser 278019 has rated the following books:\n\n""U.S.S. Seawolf"" 6\n""Ancient Blood: Day of Honor #1 (Star Trek The Next Generation)"" 9\n""Her Klingon Soul (Star Trek Voyager: Day of Honor, Book 3)"" 9\n""Treaty's Law (Star Trek: Day of Honor, Book 4)"" 9\n\nPredict the user's rating on ""Armageddon Sky (Star Trek Deep Space Nine: Day of Honor, Book 2)"". Output the rating score only. Do not include other text.","""Armageddon Sky (Star Trek Deep Space Nine: Day of Honor, Book 2)""",9
1999,"You are a book recommender and your job is to predict a user's rating (ranging from 1 to 10, with 10 being the highest) on a book, based on that user's previous ratings.\n\nUser 278565 has rated the following books:\n\n""Gilbert: A Comedy of Manners"" 8\n""Murder at the National Cathedral (Capital Crime Mysteries)"" 9\n""Glitz"" 7\n""Di and I"" 8\n\nPredict the us

In [99]:
from openai import OpenAI
client = OpenAI(api_key = 'YOUR-API-KEY')

response = client.chat.completions.create(
  model="gpt-3.5-turbo", 
  messages=[
    {"role": "system", "content": "You are a book recommender."},
    {"role": "user", "content": "Who won the world series in 2020?"},
    {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
    {"role": "user", "content": "Where was it played?"}
  ]
)

In [ ]:
# int(response.choices[0].message.content.strip("'"))
# int(df_llm_5_items['ans'][2])

In [131]:
total, false, true = 0,0,0
for idx, row in df_llm_5_items.iterrows():
    try:
        response = client.chat.completions.create(
        model="gpt-3.5-turbo", 
        messages=[
            {"role": "system", "content": "You are a book recommender."},
            {"role": "user", "content": row['prompt']},
        ]
        )
        pred = int(response.choices[0].message.content.strip("'"))
        ans = int(df_llm_5_items['ans'][2])
        if pred != ans:
            false += 1
            print(f"PROMPT {row['prompt']}")
            print(f'pred {pred} ans {ans} total false {false}th')
        else:
            true += 1
        total+=1
    except Exception as e:
        print(f"Skipping index {idx} prompt {row['prompt']} reponse {response.choices[0].message.content} due to error: {e}")
print(f'total {total} # of wrongs {false} acc {true/total*100.00}')    

PROMPT You are a book recommender and your job is to predict a user's rating (ranging from 1 to 10, with 10 being the highest) on a book, based on that user's previous ratings.

User 183 has rated the following books:

"Folio Junior: L'histoire De Monsieur Sommer" 8
"Fahrenheit 451" 9
"Cuentos del Planeta Tierra" 7
"Que Se Mueran Los Feos (Fabula)" 9

Predict the user's rating on "Estudios sobre el amor". Output the rating score only. Do not include other text.
pred 9 ans 7 total false 1th
PROMPT You are a book recommender and your job is to predict a user's rating (ranging from 1 to 10, with 10 being the highest) on a book, based on that user's previous ratings.

User 387 has rated the following books:

"The Adventures of Tom Sawyer" 9
"Santa Brought A Son : Marrying The Boss's Daughter (Silhouette Romance, 1698)" 6
"Alice's Adventures in Wonderland and Through the Looking Glass" 10
"Julius Caesar (Oxford School Shakespeare)" 2

Predict the user's rating on "Jane Eyre". Output the rat